In [1]:
!pip install langchain text2vec

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.2 MB/s eta 0:00:00:00:01
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.4/381.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 32.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━

In [2]:
import json
with open('workflow_asset/catogory_sample.json', 'r') as f:
    category = json.loads(f.read())
    
with open('workflow_asset/catogory_sample_generate.json', 'r') as f:
    category_generate = json.loads(f.read())

In [3]:
import pandas as pd

In [4]:
def generate_sample():
    
    for c, v in category.items():
        for product_sample in v:
            yield c, product_sample

        for product_sample in category_generate[c]:
            yield c, product_sample

In [5]:
df = pd.DataFrame(generate_sample(), columns=['category', 'sample'])
df['cs'] = df['category']+'-'+df['sample']

In [6]:
from langchain.vectorstores import FAISS, Chroma

In [ ]:
!pip install faiss-cpu

In [ ]:
from text2vec import SentenceModel
model = SentenceModel('shibing624/text2vec-base-chinese')
embeddings = model.encode(df['sample'])

In [ ]:
# from langchain.vectorstores.utils import DistanceStrategy
# from langchain.schema import Document
# docs = list(map(lambda x: Document(page_content=x[1]['cs'], metadata=x[1]), df.iterrows()))
# vdb = FAISS.from_embeddings(docs, embeddings, normalize_L2=True, distance_strategy=DistanceStrategy.COSINE)

In [ ]:
from typing import Any, Dict, List, Optional
from text2vec import SentenceModel

class EmbeddingModel():
    def __init__(self, model='shibing624/text2vec-base-chinese', **kwargs):

        self.embedding_model = SentenceModel()


    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute doc embeddings using a Bedrock model.

        Args:
            texts: The list of texts to embed.

        Returns:
            List of embeddings, one for each text.
        """
        return self.embedding_model.encode(texts)

    def embed_query(self, text: str) -> List[float]:
        """Compute query embeddings using a Bedrock model.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """
        return self.embedding_model.encode([text])[0]


In [ ]:
from langchain.schema import Document
from langchain.vectorstores.utils import DistanceStrategy

In [ ]:
match_col = 'sample' # cs, sample, category
docs = list(map(lambda x: Document(page_content=x[1][match_col], metadata=x[1]), df.iterrows()))
vdb = FAISS.from_documents(docs, EmbeddingModel(), normalize_L2=False, distance_strategy=DistanceStrategy.COSINE)

In [ ]:
df_samples = pd.read_csv('data/samples.csv', index_col=0)
df_samples = df_samples[df_samples['item_name'].apply(lambda x: '外送費' not in x)]
df_samples = df_samples[df_samples['item_name'].apply(lambda x: '平台費' not in x)]
df_samples = df_samples[df_samples['category'].apply(lambda x: '其他' not in x)]

In [ ]:
item = df_samples.sample(1).iloc[0]
item

In [ ]:
item_name = item['item_name']
item_name

In [ ]:
vdb.similarity_search_with_score(item_name, k=10)

In [ ]:
from multiprocessing import Pool

In [391]:
p = Pool(8)

Process ForkPoolWorker-5:
Process ForkPoolWorker-2:
Process ForkPoolWorker-8:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkPoolWorker-7:
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-1:
Traceback (most recent call last):
Process ForkPoolWorker-4:
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.r

33.9 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [375]:
vdb.similarity_search_with_score([item_name, item_name], k=1)#[0][0].metadata['category']

[(Document(page_content='FlexFit 運動遮陽帽 - 融熱黑', metadata={'category': '運動機能服飾', 'sample': 'FlexFit 運動遮陽帽 - 融熱黑', 'cs': '運動機能服飾-FlexFit 運動遮陽帽 - 融熱黑'}),
  300.34512)]

In [433]:
match_col = 'category' # cs, sample, category
docs = list(map(lambda x: Document(page_content=x[1][match_col], metadata=x[1]), df.iterrows()))
vdb = FAISS.from_documents(docs, EmbeddingModel(), normalize_L2=False, distance_strategy=DistanceStrategy.COSINE)

2023-08-03 17:48:12.448 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


In [434]:
def classify(item_name):
    # return vdb.similarity_search_with_score(item_name, k=1)[0][0].metadata['category']
    return vdb.similarity_search(item_name, k=1)[0].metadata['category']

In [435]:
%%timeit
classify(test['item_name'].iloc[0])

432 ms ± 48.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [436]:
docs[0]

Document(page_content='現做咖啡飲品', metadata={'category': '現做咖啡飲品', 'sample': '全家特濃拿鐵', 'cs': '現做咖啡飲品-全家特濃拿鐵'})

In [430]:
test = df_samples.sample(10000)
category

In [ ]:
test

,item_name,item_type,category
684518,RASTO RK8 摺疊收納伸縮式充電風扇-綠,3C類,桌機/筆電/平板
456025,Little SiS. 現+預 小夏日🍧 ＃1973 質感,3C類,手機/通訊/週邊
977661,防蚊貼片 可愛 防蚊 驅蚊神器 防蚊片 防蚊植物 嬰兒防蚊貼,日用百貨類,家庭清潔用品
507626,0652023052200106,3C類,手機/通訊/週邊
615628,琨蒂絲蕾絲長統彈性絲襪-黑,服飾鞋包類,流行鞋襪
...,...,...,...
97078,【MUJI 無印良品】棉圈絨可剪裁小浴巾/中厚型/可吊掛/煙,洗浴清潔/保養類,美容保養
833983,【日本旭川】AIRFit氧活力極致涼感支撐空氣坐墊-1人座-,3C類,週邊/耗材
987717,【現貨免運】珊瑚絨抹布 廚房清潔 抹布 去汙抹布 擦車布 洗,日用百貨類,家庭清潔用品
545088,【Hush Puppies】女裝 T恤 經典立體品牌刺繡短袖,服飾鞋包類,女裝


In [432]:
test['predict_category'] = p.map(classify, test['item_name']) #test['item_name'].apply(classify)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix

In [ ]:
accuracy_score(test['category'], test['predict'])

In [ ]:
accuracy_score(test['category'], test['predict_cs'])

In [ ]:
sum(test['predict'] != test['predict_cs'])

In [ ]:
classify('無印化妝盒')

In [ ]:
test.sample(100)